In [ ]:
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import pingouin as pg
import seaborn as sns
import pprint
from scipy import stats
import statsmodels.formula.api as smf

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
title_text_metagenres_metascore_df = pd.read_hdf("./data/out/title_metasummary_metagenres_metascore_random_criteria_20240404-163802_multilabel_big.h5")
title_text_metagenres_metascore_df['meta_score'] = pd.to_numeric(title_text_metagenres_metascore_df['meta_score'])

In [ ]:
title_text_metagenres_metascore_df['meta_score'].hist()

In [ ]:
df_z = title_text_metagenres_metascore_df.select_dtypes(include=[np.number]).dropna().apply(stats.zscore)

start = 'Q("'
end = '")'
high_low_inference_criteria_qued = ["{}{}{}".format(start,i,end) for i in list(title_text_metagenres_metascore_df.columns)[4:]]

independent_variables_formula = '+'.join(high_low_inference_criteria_qued)

# fitting regression
formula = 'Q("meta_score") ~ ' + independent_variables_formula
result = smf.ols(formula, data=df_z).fit()

# checking results
print(result.summary())

In [ ]:
list(title_text_metagenres_metascore_df.columns)[4:]

In [ ]:
ax = sns.regplot(data=title_text_metagenres_metascore_df, x='real', y='meta_score')

In [ ]:
title_text_metagenres_metascore_df.corr(numeric_only=True)['meta_score'][1:]

In [ ]:
title_text_metagenres_metascore_df[title_text_metagenres_metascore_df['anthropogenic']<0.1]

In [ ]:
meta_genres = title_text_metagenres_metascore_df["meta_genres"].explode().unique()
meta_genres = meta_genres[~pd.isnull(meta_genres)]

In [ ]:
meta_genres

In [ ]:
dict_of_genre_dfs = {}
for genre in meta_genres:
    genre_df = title_text_metagenres_metascore_df[
        title_text_metagenres_metascore_df.meta_genres.apply(lambda x: genre in x)]
    dict_of_genre_dfs[genre] = genre_df

In [ ]:
for criterion in list(title_text_metagenres_metascore_df.columns)[4:]:
    display(criterion)
    for genre, genre_df in dict_of_genre_dfs.items():
        try:
            display(genre, pg.corr(genre_df[criterion], genre_df['meta_score']).round(3))
        except ValueError:
            pass

In [ ]:
dict_of_genre_dfs['Action'].sort_values('meta_score', ascending=False)

In [ ]:
dict_of_genre_dfs['Sci-Fi'].sort_values('meta_score', ascending=False)